# Subgroups and Lagrange's Theorem

**Module 01** | 01-modular-arithmetic-groups

*Subgroup detection, Lagrange verification*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **subgroups and lagrange's theorem**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Cyclic Groups and Generators](01d-cyclic-groups-generators.ipynb).
- Concepts and notation introduced in the previous notebook.

## Finding Subgroups

In [ ]:
# List all subgroups of Z/12Z (additive)
G = Zmod(12)
# Generate subgroup from element 4
elem = G(4)
subgroup = set()
x = elem
while x not in subgroup:
    subgroup.add(x)
    x = x + elem
print(sorted(subgroup))

## Lagrange's Theorem

In [ ]:
# Verify that subgroup order divides group order
group_order = 12
for d in divisors(group_order):
    print(f'{d} divides {group_order}: {group_order % d == 0}')

## Cosets and Counting

In [ ]:
# Compute cosets of a subgroup in Z/12Z
# TODO: build cosets and verify equal partition

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **subgroups and lagrange's theorem**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Visualizing Group Structure](01f-group-visualization.ipynb)